# GraphQL Erklärungen und Einsichten

## Fields and Types
GraphQl unterscheidet im wesentlichen zwischen Types und Fields. Dabei gilt
- Types können 0 bis n Fields haben
- Fields können können (sofern im backend implementiert) Argumente bekommen und geben entsprechend Types zurück
- Man kann sich die GraphQL Datendarstellung wie folgt denken ...->Type->Field->Type->Field...
- Startpunkt ist der type *Query*
- Ende elementare Typen ohne Felder, z.B. Int, String

Folgendes beispiel bezeichnet alle komponenten:

```
{
  region(id:"05911") {
    id          
    name
        BEVMK3 {
    value
    year
    }
  }
}
```

Erklärungen zu den Zeilen:
- Zeile 3: `region` ist ein **Field** des implizit zugrundeligenden **Type** `Query`. Das Field wird mit dem **Arg** `id` aufgerufen.
- Zeile 4: `id` ist ein **Field** des Rückgabetypen von Query.region(id: ...) (dieser ist `Region`). Insbesondere ist es damit technisch nicht das Selbe wie das argument `id` in der vorhergehenden Zeile (inhaltlich sind sie gleich)
- Zeile 5 `name` ist ein **Field** des Rückgabetypen von Query.region(id: ...).
- Zeile 6 `BEVMK3` ist ein **Field** des Rückgabetypen von Query.region(id: ...). Hier bekinnt im gegensatzt zu den beiden vohergehenden Zeilen eine weitere Hierarchieebene los. Grund dafür ist dass der Rückgabewert von Region.BEVMK3 ein komplexer datentyp ist der selbst wieder Fields beinhaltet. `id` und `name` hingeben geben die elementaren Datentypen Int und String zurück. **Wichtig** in der datenguide Konvention sind hat der Typ **Region** Fields zu allen Statistiken, die wiederum Objekte vom Typ mit namen der Statistik zurückgeben. In bezug auf das aktuelle Beispiel bedeutet dies es gibt das **Field** BEVMK3 welches den **Type** BEVMK3 zurückgibt.
- Zeile 7 & 8 sind **Fields** des Rückgabewertes von Region.BEVMK3 (welcher selbst auch den Namen BEVMK3 trägt)

## Konsequenzen

Als konsequenz bedutet dies, dass man zum Abfragen von möglichen Ausgabewerten nach Fields schauen muss, und zu Abfrage von möglichen Filterwerten nach Args von Fields. Zusammengefasst
- Ausgabewerte = Fields
- Filterparameter = Args

Beispielqueries an die Datenbank sehen wie folgt aus

In [1]:
import requests
from pprint import pprint

def runQuery(queryString):
    post_json = dict()
    post_json["query"] = queryString
    header = { 'Content-Type': 'application/json' }
    URL = "https://api-next.datengui.de/graphql"
    resp = requests.post(url=URL,headers=header,json=post_json)
    
    if resp.status_code == 200:
        return resp.json()
    else:
        raise Exception(f"Http error: status code {resp.status_code}")
        
arguments_query = """
{
  __type(name: "Region") {
    fields {
      name
      args {
        name
        description
        defaultValue
        type {
          ofType {
            name
          }
        }
        }
      }
    }
}
"""
pprint([field for field in runQuery(arguments_query).get('data').get('__type').get('fields')
 if field['name'] == 'BEVMK3'][0])

{'args': [{'defaultValue': None,
           'description': 'Jahr des Stichtages',
           'name': 'year',
           'type': {'ofType': {'name': 'Int'}}},
          {'defaultValue': None,
           'description': 'Statistik',
           'name': 'statistics',
           'type': {'ofType': {'name': 'BEVMK3Statistics'}}}],
 'name': 'BEVMK3'}


In [2]:
field_query = """
{
  __type(name: "BEVMK3") {
    fields {
      name
    }
}
}
"""
pprint(runQuery(field_query)) #.get('data').get('__type').get('fields')

{'data': {'__type': {'fields': [{'name': 'id'},
                                {'name': 'year'},
                                {'name': 'value'},
                                {'name': 'source'}]}}}


## Other points worth explaining
- variables
- @inlcude/@skip
- Fragments